# Import

In [32]:
import os
import re
import sys
from collections import defaultdict

import pandas as pd

# Declare Path

In [33]:
root_dir = '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/'
dirs = [x for x in os.listdir(root_dir) if x != '.DS_Store']
files = {}

# for d in dirs:
d = 'd1_scholarship'
dir_path = os.path.join(root_dir, d)
files[d] = [os.path.join(root_dir, d, x) for x in os.listdir(dir_path) if x != '.DS_Store' and x.endswith('.tsv')]

files

{'d1_scholarship': ['/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/d1_scholarship/2016_scholarship.tsv',
  '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/d1_scholarship/2018_scholarship.tsv',
  '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/d1_scholarship/2017_scholarship.tsv']}

# Load Files

In [34]:
pd.set_option('display.max_row', 50)
df = defaultdict(dict)
for dir_name, file_list in files.items():
    key = dir_name.split('_')[-1]
    for file in file_list:
        file_name = re.search(r'(\d{4})', file.split('/')[-1]).group(1)
        df[file_name] = pd.read_csv(file, sep='\t')

df

defaultdict(dict,
            {'2016':        연도       학교종류 설립구분  지역  상태               학교명 재학생 (A)       교외 소계  \
             0    2015        대학교   사립  경북  기존  가야대학교(고령) _제2캠퍼스       1           0   
             1    2015        대학교   사립  경남  기존     가야대학교(김해) _본교   2,545   5,833,504   
             2    2015        대학교   사립  경기  기존         가천대학교 _본교  18,333  27,532,517   
             3    2015        대학교   사립  강원  기존      가톨릭관동대학교 _본교   8,809  16,715,770   
             4    2015        대학교   사립  경기  기존        가톨릭대학교 _본교   7,115  12,275,031   
             5    2015        대학교   사립  서울  기존     가톨릭대학교 _제2캠퍼스     451     824,243   
             6    2015        대학교   사립  서울  기존     가톨릭대학교 _제3캠퍼스     182     290,086   
             7    2015        대학교   사립  서울  기존      감리교신학대학교 _본교     844   2,286,304   
             8    2015        대학교   사립  경기  기존         강남대학교 _본교   6,955  14,000,485   
             9    2015        대학교   국립  강원  기존       강릉원주대학교 _본교   5,964  12,497,432   
      

# Pre-process

In [35]:
# df['2016_submission'][cols_sub] = df['2016_submission'][cols_sub].applymap(lambda x: pd.to_numeric(x.replace(',', '') if type(x)!=int else x))
convert_str2int(df['2016'], cols_numeric)
convert_str2int(df['2017'], cols_numeric)
convert_str2int(df['2018'], cols_numeric)

# Functions

In [36]:
def convert_str2int(d, cols):
    d[cols] = d[cols].applymap(lambda x: pd.to_numeric(x.replace(',', '') if type(x)!=int else x))

# Index and Columns

In [37]:
df['2018'].columns

Index(['연도', '학교종류', '설립구분', '지역', '상태', '학교명', '재학생 (A)', '교외 소계', '교외 국가',
       '교외 지방자치단체', '교외 사설 및 기타', '교내 소계', '교내 성적우수 장학금', '교내 저소득층 장학금',
       '교내 근로 장학금', '교내 교직원 장학금', '교내 기타', '총계(B)', '교내외국인유학생(외국인전형,C)',
       '재학생 1인당 장학금 (D=B/A)'],
      dtype='object')

In [51]:
# 13 univ
univ_names = ['서울', '연세', '고려', '서강', '성균관', '한양', '중앙', '경희', '한국외국어', '서울시립', '건국', '동국', '홍익']
univ = [u + '대학교 _본교' for u in univ_names]

idx_2016 = df['2016']['학교명'].isin(univ)
idx_2017 = df['2017']['학교명'].isin(univ)
idx_2018 = df['2018']['학교명'].isin(univ)

idx_2018_above = df['2018']['재학생 (A)'] >= 10000
idx_2018_not_cyber = df['2018']['학교종류'] == '대학교'
idx_2018_seoul = df['2018']['지역'] == '서울'
idx_2018 = idx_2018_above & idx_2018_not_cyber & idx_2018_seoul

idx_public1_2018 = df['2018']['설립구분'].str.contains('국립')
idx_public2_2018 = df['2018']['설립구분'].str.contains('공립')
idx_public3_2018 = df['2018']['설립구분'].str.contains('법인')

idx_public_2018 = idx_public1_2018 | idx_public2_2018 | idx_public3_2018

cols = []

cols_numeric = [
    '재학생 (A)', '교외 소계', '교외 국가',
       '교외 지방자치단체', '교외 사설 및 기타', '교내 소계', '교내 성적우수 장학금', '교내 저소득층 장학금',
       '교내 근로 장학금', '교내 교직원 장학금', '교내 기타', '총계(B)', '교내외국인유학생(외국인전형,C)',
       '재학생 1인당 장학금 (D=B/A)'
]

# Export_TSV

In [48]:
# name0 = 'd1-7_scholarship_per_student.tsv'
# file0 = os.path.join(root_dir, name0)
# df['2018'].loc[idx_2018].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False).to_csv(file0, sep='\t', index=False)

# name1 = 'd1-8_scholarship_all.tsv'
# file1 = os.path.join(root_dir, name1)
# df['2018'].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False)[:10].to_csv(file1, sep='\t', index=False)

In [40]:
# name1 = 'd1-12_scholarship_public.tsv'
# file1 = os.path.join(root_dir, name1)
# df['2018'].loc[idx_public_2018].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False)[:10].to_csv(file1, sep='\t', index=False)

# -------------------------------------------------------------------

In [52]:
df['2018'].loc[idx_2018].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False)

,연도,학교종류,설립구분,지역,상태,학교명,재학생 (A),교외 소계,교외 국가,교외 지방자치단체,교외 사설 및 기타,교내 소계,교내 성적우수 장학금,교내 저소득층 장학금,교내 근로 장학금,교내 교직원 장학금,교내 기타,총계(B),"교내외국인유학생(외국인전형,C)",재학생 1인당 장학금 (D=B/A)
251,2017,대학교,사립,서울,기존,홍익대학교 _본교,11771,17773205,17037385,55169,680651,27101760,13656641,6405899,3001304,85568,3952348,44874965,227171,3812.3
129,2017,대학교,사립,서울,기존,세종대학교 _본교,11336,22341902,22048657,23454,269791,19961546,2829693,5614216,667014,34034,10816589,42303448,2239639,3731.8
126,2017,대학교,사립,서울,기존,성균관대학교 _본교,18597,25423394,23486031,30459,1906904,39139481,18108551,10649672,2399754,4557,7976947,64562875,3191764,3471.7
171,2017,대학교,사립,서울,기존,이화여자대학교 _본교,15016,25137259,22145387,72489,2919383,25699751,8092894,7950246,769226,186568,8700817,50837010,1203574,3385.5
242,2017,대학교,사립,서울,기존,한양대학교 _본교,15117,25789511,21976876,321237,3491398,24021773,3824807,6137008,976310,224062,12859586,49811284,860186,3295.1
139,2017,대학교,사립,서울,기존,숭실대학교 _본교,12654,22316947,19677510,251237,2388200,19313299,4379937,5319170,1394295,99425,8120472,41630246,2043761,3289.9
150,2017,대학교,사립,서울,기존,연세대학교 _본교,18690,31364638,25307349,381700,5675589,29244680,7029077,13292027,2943112,852770,5127694,60609318,1896370,3242.9
12,2017,대학교,사립,서울,기존,건국대학교 _본교,14980,25513247,23430041,176361,1906845,22597143,7484090,9288253,867937,210962,4745901,48110390,1677267,3211.6
49,2017,대학교,사립,서울,기존,국민대학교 _본교,14577,25885187,22865396,161494,2858297,20066046,5116866,4595985,1829879,80691,8442625,45951233,2175734,3152.3
114,2017,대학교,국립대법인,서울,기존,서울대학교 _본교,16158,36148747,23588063,129907,12430777,14211985,7110994,3012482,3387505,0,701004,50360732,110890,3116.8


In [42]:
df['2018'].loc[idx_2018].sort_values('재학생 (A)', ascending=False)

,연도,학교종류,설립구분,지역,상태,학교명,재학생 (A),교외 소계,교외 국가,교외 지방자치단체,교외 사설 및 기타,교내 소계,교내 성적우수 장학금,교내 저소득층 장학금,교내 근로 장학금,교내 교직원 장학금,교내 기타,총계(B),"교내외국인유학생(외국인전형,C)",재학생 1인당 장학금 (D=B/A)
33,2017,대학교,사립,서울,기존,경희대학교 _본교,24441,38042564,34225746,354055,3462763,35184085,8124585,14122633,644917,327366,11964584,73226649,2125229,2996.1
36,2017,대학교,사립,서울,기존,고려대학교 _본교,20468,31012552,24334354,204151,6474047,30091568,1947046,15081014,3695551,487122,8880835,61104120,1345681,2985.3
150,2017,대학교,사립,서울,기존,연세대학교 _본교,18690,31364638,25307349,381700,5675589,29244680,7029077,13292027,2943112,852770,5127694,60609318,1896370,3242.9
126,2017,대학교,사립,서울,기존,성균관대학교 _본교,18597,25423394,23486031,30459,1906904,39139481,18108551,10649672,2399754,4557,7976947,64562875,3191764,3471.7
189,2017,대학교,사립,서울,기존,중앙대학교 _본교,17047,25884847,22966110,132982,2785755,24601536,8330965,10991613,1127491,186702,3964765,50486383,1015348,2961.6
225,2017,대학교,사립,서울,기존,한국외국어대학교 _본교,16358,31220766,28742806,73706,2404254,17910162,5029127,5131062,1591865,177234,5980874,49130928,1553596,3003.5
114,2017,대학교,국립대법인,서울,기존,서울대학교 _본교,16158,36148747,23588063,129907,12430777,14211985,7110994,3012482,3387505,0,701004,50360732,110890,3116.8
242,2017,대학교,사립,서울,기존,한양대학교 _본교,15117,25789511,21976876,321237,3491398,24021773,3824807,6137008,976310,224062,12859586,49811284,860186,3295.1
171,2017,대학교,사립,서울,기존,이화여자대학교 _본교,15016,25137259,22145387,72489,2919383,25699751,8092894,7950246,769226,186568,8700817,50837010,1203574,3385.5


In [43]:
df['2018'].loc[idx_public_2018].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False)

,연도,학교종류,설립구분,지역,상태,학교명,재학생 (A),교외 소계,교외 국가,교외 지방자치단체,교외 사설 및 기타,교내 소계,교내 성적우수 장학금,교내 저소득층 장학금,교내 근로 장학금,교내 교직원 장학금,교내 기타,총계(B),"교내외국인유학생(외국인전형,C)",재학생 1인당 장학금 (D=B/A)
214,2017,대학교,특별법법인,대전,기존,한국과학기술원 _본교,3815,6380990,6252501,1690,126799,22169330,1704286,117160,348655,2400,19996829,28550320,2084182,7483.7
45,2017,대학교,특별법법인,광주,기존,광주과학기술원 _본교,714,576995,559995,0,17000,3881426,107716,111446,18414,0,3643850,4458421,0,6244.3
163,2017,대학교,특별법법인,울산,기존,울산과학기술원 _본교,2769,6593713,6166862,0,426851,10405629,6677870,182369,120571,0,3424819,16999342,1652636,6139.2
46,2017,교육대학,국립,광주,기존,광주교육대학교 _본교,1379,3806636,3186033,470424,150179,1241973,245289,153990,339889,0,502805,5048609,0,3661.1
11,2017,대학교,국립,강원,기존,강원대학교 _제2캠퍼스,5917,15726615,12239010,3154102,333503,5526341,2146437,1486270,448775,0,1444859,21252956,47974,3591.8
216,2017,대학교,국립,충북,기존,한국교통대학교 _본교,8191,22223013,21951813,43616,227584,6769034,3719591,676030,1098390,0,1275023,28992047,27304,3539.5
176,2017,대학교,국립대법인,인천,기존,인천대학교 _본교,11689,25835610,25072131,52775,710704,10787194,4030852,2731274,1497437,0,2527631,36622804,395990,3133.1
8,2017,대학교,국립,강원,기존,강릉원주대학교 _본교,5621,12890050,12259561,64519,565970,4667832,2597646,1100972,374111,0,595103,17557882,75988,3123.6
193,2017,교육대학,국립,경남,기존,진주교육대학교 _본교,1362,3173145,3051901,0,121244,1072230,270102,209159,77177,0,515792,4245375,0,3117.0
114,2017,대학교,국립대법인,서울,기존,서울대학교 _본교,16158,36148747,23588063,129907,12430777,14211985,7110994,3012482,3387505,0,701004,50360732,110890,3116.8


In [44]:
df['2018'].loc[df['2018']['지역']=='서울'].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False)

,연도,학교종류,설립구분,지역,상태,학교명,재학생 (A),교외 소계,교외 국가,교외 지방자치단체,교외 사설 및 기타,교내 소계,교내 성적우수 장학금,교내 저소득층 장학금,교내 근로 장학금,교내 교직원 장학금,교내 기타,총계(B),"교내외국인유학생(외국인전형,C)",재학생 1인당 장학금 (D=B/A)
5,2017,대학교,사립,서울,기존,가톨릭대학교 _제3캠퍼스,170,288807,240379,0,48428,784851,25872,47638,0,0,711341,1073658,48187,6315.6
222,2017,대학교,사립,서울,기존,한국성서대학교 _본교,1050,3906038,3791488,64760,49790,1424210,331129,161213,369213,18176,544479,5330248,117864,5076.4
125,2017,대학교,사립,서울,기존,성공회대학교 _본교,2049,6126531,6037523,9000,80008,2992948,716204,1070072,342914,61463,802295,9119479,4345,4450.7
104,2017,대학교,사립,서울,기존,삼육대학교 _본교,5381,14857293,14166976,223015,467302,8140227,1625748,1746970,1702951,185827,2878731,22997520,46342,4273.8
113,2017,대학교,사립,서울,기존,서울기독대학교 _본교,547,808889,773205,0,35684,1485619,183215,857838,364623,14408,65535,2294508,0,4194.7
127,2017,대학교,사립,서울,기존,성신여자대학교 _본교,8916,19630277,18899129,64657,666491,17397073,8008306,4456170,509075,22194,4401328,37027350,22444,4152.9
178,2017,대학교,사립,서울,기존,장로회신학대학교 _본교,648,2075849,1781197,105347,189305,561351,119698,140099,66438,0,235116,2637200,5412,4069.8
202,2017,대학교,사립,서울,기존,총신대학교 _본교,1848,5784043,5250578,0,533465,1735005,367195,678043,305306,28228,356233,7519048,35438,4068.7
121,2017,대학교,사립,서울,기존,서울한영대학교 _본교,683,1956155,1876924,16000,63231,751514,159412,80643,135795,1915,373749,2707669,0,3964.4
239,2017,대학교,사립,서울,기존,한성대학교 _본교,6125,14726058,14061718,87691,576649,9317654,2460210,1916221,1673905,50468,3216850,24043712,125651,3925.5


In [45]:
df['2018'].loc[idx_2018, '재학생 1인당 장학금 (D=B/A)'].mean()

3162.0555555555557

In [46]:
df['2018'].sort_values('재학생 1인당 장학금 (D=B/A)', ascending=False)

,연도,학교종류,설립구분,지역,상태,학교명,재학생 (A),교외 소계,교외 국가,교외 지방자치단체,교외 사설 및 기타,교내 소계,교내 성적우수 장학금,교내 저소득층 장학금,교내 근로 장학금,교내 교직원 장학금,교내 기타,총계(B),"교내외국인유학생(외국인전형,C)",재학생 1인당 장학금 (D=B/A)
133,2017,대학교,사립,경기,기존,수원가톨릭대학교 _본교,120,430959,211030,0,219929,577826,32620,11600,0,0,533606,1008785,0,8406.5
214,2017,대학교,특별법법인,대전,기존,한국과학기술원 _본교,3815,6380990,6252501,1690,126799,22169330,1704286,117160,348655,2400,19996829,28550320,2084182,7483.7
174,2017,대학교,사립,인천,기존,인천가톨릭대학교 _본교,79,20317,18317,0,2000,558582,12100,51120,0,0,495362,578899,0,7327.8
149,2017,대학교,사립,인천,기존,안양대학교 _제2캠퍼스,202,664531,657906,1000,5625,683696,255994,215193,73931,8450,130128,1348227,0,6674.4
54,2017,대학교,사립,충남,기존,금강대학교 _본교,514,867621,860050,0,7571,2541821,1595025,24585,78319,0,843892,3409442,3282,6633.2
57,2017,대학교,사립,충북,기존,꽃동네대학교 _본교,452,1797300,1715884,2000,79416,1113086,290292,260651,256313,0,305830,2910386,23388,6438.9
5,2017,대학교,사립,서울,기존,가톨릭대학교 _제3캠퍼스,170,288807,240379,0,48428,784851,25872,47638,0,0,711341,1073658,48187,6315.6
45,2017,대학교,특별법법인,광주,기존,광주과학기술원 _본교,714,576995,559995,0,17000,3881426,107716,111446,18414,0,3643850,4458421,0,6244.3
163,2017,대학교,특별법법인,울산,기존,울산과학기술원 _본교,2769,6593713,6166862,0,426851,10405629,6677870,182369,120571,0,3424819,16999342,1652636,6139.2
102,2017,대학교,사립,경남,기존,부산장신대학교 _본교,240,1131858,1099607,0,32251,268795,23293,46160,33126,9006,157210,1400653,4112,5836.1


In [47]:
df['2018'].loc[~idx_2018, '재학생 1인당 장학금 (D=B/A)'].mean()

3388.2228571428564